# 05. Date and time in Spark data frames

Date and time column are essential for time series data. In spark, we can express date and time with **four different data types**: 
- String : Most commun format, no compability issues. But need to be parsed to long or timestamp types to do arithmetic operations
- Long : in spark, it represents **unix_timestamp** implementation. it uses seconds since the UNIX epoch
- timestamp :  spark has its own implementation to represent timestamp. Represents values comprising values of fields year, month, day, hour, minute, and second, with the **session local time-zone**. The timestamp value represents an absolute point in time.
- date: Represents values comprising values of fields year, month and day, without a time-zone.

Below, we will show how to do convertion between them 

In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import lit, unix_timestamp, col, from_unixtime, to_timestamp, date_format, to_date

In [2]:
local = True

if local:
    spark=SparkSession.builder.master("local[4]").appName("DateAndTime").getOrCreate()
else:
    spark = SparkSession \
    .builder.master("k8s://https://kubernetes.default.svc:443") \
    .appName("SparkArrowCompression") \
    .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:master") \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT']) \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.memory","8g") \
    .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE']) \
    .getOrCreate()

data = [("2019-07-01 12:01:19", "07-01-2019 12:01:19.888", "07-01-2019"),
            ("2018-07-01 12:01:19", "07-01-2018 12:01:19.666", "07-01-2018"),
            ("2017-07-01 12:01:19", "07-01-2017 12:01:19.111", "07-01-2017")]
columns = ["timestamp_1", "timestamp_2", "timestamp_3"]

In [5]:
df=spark.createDataFrame(data=data,schema=columns)
df.show()
df.printSchema()

+-------------------+--------------------+-----------+
|        timestamp_1|         timestamp_2|timestamp_3|
+-------------------+--------------------+-----------+
|2019-07-01 12:01:19|07-01-2019 12:01:...| 07-01-2019|
|2018-07-01 12:01:19|07-01-2018 12:01:...| 07-01-2018|
|2017-07-01 12:01:19|07-01-2017 12:01:...| 07-01-2017|
+-------------------+--------------------+-----------+

root
 |-- timestamp_1: string (nullable = true)
 |-- timestamp_2: string (nullable = true)
 |-- timestamp_3: string (nullable = true)



## 5.1 Convert string date, timestamp to long (seconds since the UNIX epoch)

To convert string to long, we can use unix_timestamp() function, it takes a string date, and a date format, if the date format is not provided, the default format (yyyy-MM-dd HH:mm:ss) will be used. Then the string date are converted to Unix timestamp (in seconds) by using the current timezone of the system. 

Note you can set the timezone of your spark session by using **spark.conf.set("spark.sql.session.timeZone", "<time_zone>")**, for example spark.conf.set("spark.sql.session.timeZone", "US/Pacific").

1: def unix_timestamp(): returns the current time in seconds (LongType)
2) def unix_timestamp(date: Column): take the string column date as input, output it in seconds, it will use default date format (yyyy-MM-dd HH:mm:ss) to parse the string. 
3) def unix_timestamp(date: Column, format: String): we can specify explicitly the format of the date string, if the default format does not fit.

In [7]:
# We add a new column that is filled with the current time
# note the column type
df1 = df.withColumn("timestamp_4", lit(unix_timestamp()))
df1.printSchema()
df1.show()


root
 |-- timestamp_1: string (nullable = true)
 |-- timestamp_2: string (nullable = true)
 |-- timestamp_3: string (nullable = true)
 |-- timestamp_4: long (nullable = true)

+-------------------+--------------------+-----------+-----------+
|        timestamp_1|         timestamp_2|timestamp_3|timestamp_4|
+-------------------+--------------------+-----------+-----------+
|2019-07-01 12:01:19|07-01-2019 12:01:...| 07-01-2019| 1632594547|
|2018-07-01 12:01:19|07-01-2018 12:01:...| 07-01-2018| 1632594547|
|2017-07-01 12:01:19|07-01-2017 12:01:...| 07-01-2017| 1632594547|
+-------------------+--------------------+-----------+-----------+



In [8]:
# note the output dataframe has timestamp 1 and timestamp 2 correct. But timestamp 3 wrong. 
# we did not give format for timestamp_1 and timestamp_3, it works for timestamp_1 because it uses the default format is yyyy-MM-dd HH:mm:ss
# timestamp_3 does not work, because it does not use the default date format.
# for timestamp_2, we specify a date format, if we don't give the format, it returns null, because the string does not have
# the default date format
# If we give a format that is wrong, the spark job will fail. For example, try to remove SSS from timestamp_2 format, and see what happens

df2 = df.withColumn("timestamp_1", unix_timestamp("timestamp_1")) \
        .withColumn("timestamp_2", unix_timestamp("timestamp_2", "MM-dd-yyyy HH:mm:ss.SSS")) \
        .withColumn("timestamp_3", unix_timestamp("timestamp_3"))
print("Exp1: covert date string to seconds with some erreurs")
df2.printSchema()
df2.show()

Exp1: covert date string to seconds with some erreurs
root
 |-- timestamp_1: long (nullable = true)
 |-- timestamp_2: long (nullable = true)
 |-- timestamp_3: long (nullable = true)

+-----------+-----------+-----------+
|timestamp_1|timestamp_2|timestamp_3|
+-----------+-----------+-----------+
| 1561982479| 1561982479|       null|
| 1530446479| 1530446479|       null|
| 1498910479| 1498910479|       null|
+-----------+-----------+-----------+



In [10]:
# below, we give each one a correct date format.
df3 = df.withColumn("timestamp_1", unix_timestamp("timestamp_1")) \
        .withColumn("timestamp_2", unix_timestamp("timestamp_2", "MM-dd-yyyy HH:mm:ss.SSS")) \
        .withColumn("timestamp_3", unix_timestamp("timestamp_3", "MM-dd-yyyy")) \
        .withColumn("timestamp_4", lit(unix_timestamp()))
print("Exp1: covert date string to seconds with success")
df3.printSchema()
df3.show()

Exp1: covert date string to seconds with success
root
 |-- timestamp_1: long (nullable = true)
 |-- timestamp_2: long (nullable = true)
 |-- timestamp_3: long (nullable = true)
 |-- timestamp_4: long (nullable = true)

+-----------+-----------+-----------+-----------+
|timestamp_1|timestamp_2|timestamp_3|timestamp_4|
+-----------+-----------+-----------+-----------+
| 1561982479| 1561982479| 1561939200| 1632594849|
| 1530446479| 1530446479| 1530403200| 1632594849|
| 1498910479| 1498910479| 1498867200| 1632594849|
+-----------+-----------+-----------+-----------+



## 5.2 Convert long(seconds since the UNIX epoch) back to string date

We can use from_unixtime(unix_time: Column, format: String) to convert long back to string


In [13]:
# Convert unix timestamp in second to string date
df4 = df3.select(
        from_unixtime(col("timestamp_1")).alias("timestamp_1"),
        from_unixtime(col("timestamp_2"), "MM-dd-yyyy HH:mm:ss").alias("timestamp_2"),
        from_unixtime(col("timestamp_3"), "MM-dd-yyyy").alias("timestamp_3"),
        from_unixtime(col("timestamp_4")).alias("timestamp_4")
    )
df4.printSchema()
df4.show(truncate=False)

root
 |-- timestamp_1: string (nullable = true)
 |-- timestamp_2: string (nullable = true)
 |-- timestamp_3: string (nullable = true)
 |-- timestamp_4: string (nullable = true)

+-------------------+-------------------+-----------+-------------------+
|timestamp_1        |timestamp_2        |timestamp_3|timestamp_4        |
+-------------------+-------------------+-----------+-------------------+
|2019-07-01 12:01:19|07-01-2019 12:01:19|07-01-2019 |2021-09-25 18:39:03|
|2018-07-01 12:01:19|07-01-2018 12:01:19|07-01-2018 |2021-09-25 18:39:03|
|2017-07-01 12:01:19|07-01-2017 12:01:19|07-01-2017 |2021-09-25 18:39:03|
+-------------------+-------------------+-----------+-------------------+



## 5.3 Convert string to spark timestamp

Spark provides its own timestamp type. Instead of long, we can also use spark timestamp to store time data.

To convert a string date to spark timestamp, we can use:
- to_timestamp(col, format): the default format is yyyy-MM-dd HH:mm:ss. If it failed, return null.


In [15]:
# to_timestamp() works similar to unix_timestamp(). For timestamp_1, it has the default format, so no need to specify the format
# For 2 and 3, we need to specify the date format. Otherwise, it returns null

# note the output column type

df5 = df.withColumn("timestamp_1", to_timestamp("timestamp_1")) \
        .withColumn("timestamp_2", to_timestamp("timestamp_2", "MM-dd-yyyy HH:mm:ss.SSS")) \
        .withColumn("timestamp_3", to_timestamp("timestamp_3", "MM-dd-yyyy"))

df5.printSchema()
df5.show(truncate=False)

root
 |-- timestamp_1: timestamp (nullable = true)
 |-- timestamp_2: timestamp (nullable = true)
 |-- timestamp_3: timestamp (nullable = true)

+-------------------+-----------------------+-------------------+
|timestamp_1        |timestamp_2            |timestamp_3        |
+-------------------+-----------------------+-------------------+
|2019-07-01 12:01:19|2019-07-01 12:01:19.888|2019-07-01 00:00:00|
|2018-07-01 12:01:19|2018-07-01 12:01:19.666|2018-07-01 00:00:00|
|2017-07-01 12:01:19|2017-07-01 12:01:19.111|2017-07-01 00:00:00|
+-------------------+-----------------------+-------------------+



In [20]:
# we can also use to_timestamp in a sql 
spark.sql("select to_timestamp('06-24-2019 12:01:19.000','MM-dd-yyyy HH:mm:ss.SSSS') as timestamp")

DataFrame[timestamp: timestamp]

## 5.4 Convert spark timestamp back to string 

To convert timestamp back to string, we have two options 
- cast("String"): It converts a timestamp column to default date format yyyy-MM-dd HH:mm:ss
- date_format(column,format): It converts a timestamp column to any Java Date formats specified in DateTimeFormatter.

### 5.4.1 Use cast("String")

In [18]:
# convert spark timestamp back to string by using cast()
# note they all use the default date format.
df6 = df5.select(col("timestamp_2").cast("string"), \
                col("timestamp_2").cast("string"))
df6.printSchema()
df6.show(truncate=False)

root
 |-- timestamp_2: string (nullable = true)
 |-- timestamp_2: string (nullable = true)

+-----------------------+-----------------------+
|timestamp_2            |timestamp_2            |
+-----------------------+-----------------------+
|2019-07-01 12:01:19.888|2019-07-01 12:01:19.888|
|2018-07-01 12:01:19.666|2018-07-01 12:01:19.666|
|2017-07-01 12:01:19.111|2017-07-01 12:01:19.111|
+-----------------------+-----------------------+



### 5.4.2 Use date_format()

In [19]:
# use date_format() to convert spark timestamp to various string date
df7 = df5.withColumn("str_date_yyyy_MM_dd", date_format(col("timestamp_2"), "yyyy MM dd")) \
        .withColumn("str_date_MM/dd/yyyy_hh:mm", date_format(col("timestamp_2"), "MM/dd/yyyy hh:mm")) \
        .withColumn("str_date_yyyy_MMM_dd", date_format(col("timestamp_2"), "yyyy MMM dd")) \
        .withColumn("str_date_yyyy_MMMM_dd_E", date_format(col("timestamp_2"), "yyyy MMMM dd E"))
    
df7.printSchema()
df7.show(truncate=False)

root
 |-- timestamp_1: timestamp (nullable = true)
 |-- timestamp_2: timestamp (nullable = true)
 |-- timestamp_3: timestamp (nullable = true)
 |-- str_date_yyyy_MM_dd: string (nullable = true)
 |-- str_date_MM/dd/yyyy_hh:mm: string (nullable = true)
 |-- str_date_yyyy_MMM_dd: string (nullable = true)
 |-- str_date_yyyy_MMMM_dd_E: string (nullable = true)

+-------------------+-----------------------+-------------------+-------------------+-------------------------+--------------------+-----------------------+
|timestamp_1        |timestamp_2            |timestamp_3        |str_date_yyyy_MM_dd|str_date_MM/dd/yyyy_hh:mm|str_date_yyyy_MMM_dd|str_date_yyyy_MMMM_dd_E|
+-------------------+-----------------------+-------------------+-------------------+-------------------------+--------------------+-----------------------+
|2019-07-01 12:01:19|2019-07-01 12:01:19.888|2019-07-01 00:00:00|2019 07 01         |07/01/2019 12:01         |2019 Jul 01         |2019 July 01 Mon       |
|2018-07-01 1

## 5.5 use to_date() function to convert string to date type

In [25]:
data1=[["02-03-2013"], 
       ["05-06-2023"]]
columns1=["input"]
df8 = spark.createDataFrame(data=data1,schema=columns1 )
df9 = df8.withColumn("date", to_date("input", "MM-dd-yyyy"))

df9.printSchema()
df9.show()

root
 |-- input: string (nullable = true)
 |-- date: date (nullable = true)

+----------+----------+
|     input|      date|
+----------+----------+
|02-03-2013|2013-02-03|
|05-06-2023|2023-05-06|
+----------+----------+

